In [ ]:
# Scripts to do:
# - Refresher
# - Mapper

In [33]:
re.findall("filename=(.+)", response_head_cd)[0]

'gdc_download_20230606_101206.594610.tar.gz'

In [18]:
for file_entry in json.loads(response.content.decode("utf-8"))["data"]["hits"]:
    print(file_entry)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte

In [5]:
#!/usr/bin/env python3
# coding: utf-8

###############################################################################################################

import requests
import json
import re
import os
import glob
import tarfile

# Access the file endpoint from GDC for id retrieval
files_endpt = "https://api.gdc.cancer.gov/files"


# Filtering for TCGA Masked Somatic Mutation
# - results in open access maf files.
# This set of filters is nested under an 'and' operator.
filters = {
    "op": "and",
    "content": [
        {
            "op": "in",
            "content": {
                "field": "cases.project.program.name",
                "value": ["TCGA"]
            }
        },
        {
            "op": "in",
            "content": {
                "field": "files.data_type",
                "value": ["Masked Somatic Mutation"]
            }
        }
    ]
}

# Here a GET is used, so the filter parameters should be passed as a
# JSON string.
params = {
    "filters": json.dumps(filters),
    "fields": "file_id",
    "format": "JSON",
    "size": "20"
}

# Download the ids
response = requests.get(files_endpt, params=params)

# Create a list for the ids
file_uuid_list = []

# This step populates the download list with the file_ids from the 
# previous query
for file_entry in json.loads(response.content.decode("utf-8"))["data"]["hits"]:
    file_uuid_list.append(file_entry["file_id"])


# Create an empty list to store UUID chunks
ls = []

# Append sublists to 'ls' for 1000 ids each (server limits)
for i in range(0, len(file_uuid_list), 1000):
    ls.append(file_uuid_list[i:i + 1000])

# Download the files
for idls in ls:
    data_endpt = "https://api.gdc.cancer.gov/data"

    params = {"ids": idls}

    response = requests.post(data_endpt,
                            data=json.dumps(params),
                            headers={"Content-Type": "application/json"})

    response_head_cd = response.headers["Content-Disposition"]

    file_name = re.findall("filename=(.+)", response_head_cd)[0]

#     save_path = 'temp/'

#     completeName = os.path.join(save_path, file_name)

    with open(file_name, "wb") as output_file:

        # Open the .tar.gz file
        try:
            tar = tarfile.open(output_file, "r:gz")
        except tarfile.ReadError:
            print("Error: Invalid or corrupted .tar.gz file.")
            exit()

        # Extract all files from the archive
        try:
            tar.extractall(path="output_directory")
            print("Extraction completed.")
        except tarfile.TarError as e:
            print(f"Error extracting files from .tar.gz file: {e}")

        # Close the .tar.gz file
        tar.close()
        
#         # Extract the inner tar.gz file
#         inner_tar_data = outer_tar.extractfile('inner.tar.gz').read()

#         # Open the inner tar.gz file
#         inner_tar = tarfile.open(fileobj=io.BytesIO(inner_tar_data), mode='r:gz')

#         # Extract a file from the inner tar.gz file
#         inner_tar.extract('file.txt', path='destination_folder')

TypeError: expected str, bytes or os.PathLike object, not BufferedWriter

In [11]:
import os
import requests
import gzip
import pandas as pd

# Set the output file path
output_file = "consolidated_variants.csv"

# Set the URL for unrestricted MAF files download
url = "https://api.gdc.cancer.gov/data"

# Set the GDC filters to specify the data you want to download
filters = {
    "op": "and",
    "content": [
        {
            "op": "in",
            "content": {
                "field": "cases.project.program.name",
                "value": ["TCGA"]
            }
        },
        {
            "op": "in",
            "content": {
                "field": "files.data_type",
                "value": ["Masked Somatic Mutation"]
            }
        }
    ]
}

# Create the request payload
data = {
    "filters": filters,
    "format": "json",
    "size": 10000
}

# Send the request to retrieve the file UUIDs
response = requests.post(url, json=data)

# Check if the request was successful
if response.status_code == 200:
    # Extract the file UUIDs from the response
    response_data = response.json()
    if "data" in response_data and "hits" in response_data["data"]:
        file_uuids = [f["file_id"] for f in response_data["data"]["hits"]]

        # Download and extract variant information from each MAF file
        for file_uuid in file_uuids:
            # Construct the download URL for the MAF file
            download_url = f"{url}/{file_uuid}"

            # Send a request to download the file
            response = requests.get(download_url, stream=True)

            # Check if the request was successful
            if response.status_code == 200:
                # Get the filename from the response headers
                filename = response.headers["Content-Disposition"].split("filename=")[-1].strip()

                # Create a temporary filename for the downloaded file
                temp_filename = f"{filename}.gz"

                # Save the downloaded file to disk
                with open(temp_filename, "wb") as file:
                    file.write(response.content)

                # Extract the variant information from the MAF file
                with gzip.open(temp_filename, "rt") as file:
                    maf_data = pd.read_csv(file, sep="\t")

                # Append the variant information to the consolidated file
                maf_data.to_csv(output_file, mode="a", header=False, index=False)

                # Delete the downloaded and extracted file
                os.remove(temp_filename)

            print("Extraction completed successfully.")
            
    else:
        print("No file UUIDs found in the response.")

else:
    print("An error occurred while retrieving the file UUIDs.")

No file UUIDs found in the response.


In [8]:
maf_data

NameError: name 'maf_data' is not defined

In [2]:
# import rpy2's package module
import rpy2.robjects.packages as rpackages

# import R's utility package
utils = rpackages.importr('utils')

# select a mirror for R packages
utils.chooseCRANmirror(ind=1) # select the first mirror in the list

<rpy2.rinterface_lib.sexp.NULLType object at 0x1059ffd80> [RTYPES.NILSXP]

In [3]:
# R package names
packnames = ('ggplot2', 'hexbin', 'dplyr', 'tidyverse', 'TCGAutils', 'readr')

# R vector of strings
from rpy2.robjects.vectors import StrVector

# Selectively install what needs to be install.
# We are fancy, just because we can.
names_to_install = [x for x in packnames if not rpackages.isinstalled(x)]
if len(names_to_install) > 0:
    utils.install_packages(StrVector(names_to_install))

R[write to console]: Installing packages into ‘/opt/homebrew/lib/R/4.3/site-library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependencies ‘fastmap’, ‘sys’, ‘ps’, ‘sass’, ‘base64enc’, ‘digest’, ‘colorspace’, ‘cachem’, ‘rappdirs’, ‘rematch’, ‘askpass’, ‘prettyunits’, ‘processx’, ‘evaluate’, ‘highr’, ‘xfun’, ‘yaml’, ‘bslib’, ‘fontawesome’, ‘htmltools’, ‘jquerylib’, ‘tinytex’, ‘bit’, ‘farver’, ‘labeling’, ‘munsell’, ‘RColorBrewer’, ‘viridisLite’, ‘fansi’, ‘pkgconfig’, ‘utf8’, ‘backports’, ‘ellipsis’, ‘memoise’, ‘blob’, ‘DBI’, ‘data.table’, ‘gargle’, ‘uuid’, ‘cellranger’, ‘curl’, ‘ids’, ‘rematch2’, ‘mime’, ‘openssl’, ‘timechange’, ‘systemfonts’, ‘textshaping’, ‘progress’, ‘callr’, ‘fs’, ‘knitr’, ‘rmarkdown’, ‘selectr’, ‘stringi’, ‘bit64’, ‘cli’, ‘glue’, ‘gtable’, ‘isoband’, ‘lifecycle’, ‘rlang’, ‘scales’, ‘tibble’, ‘vctrs’, ‘withr’, ‘generics’, ‘magrittr’, ‘pillar’, ‘R6’, ‘tidyselect’, ‘broom’, ‘conflicted’, ‘dbplyr’, ‘dtplyr’, ‘forcats’, ‘googledrive’, ‘googlesh

R[write to console]: 

R[write to console]: downloaded 7833 bytes


R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/digest_0.6.31.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 163502 bytes (159 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to conso

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: 

R[write to console]: downloaded 5730 bytes


R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/prettyunits_1.1.1.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 10366 bytes (10 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to con

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: 

R[write to console]: downloaded 92 KB


R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/bslib_0.4.2.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 4830133 bytes (4.6 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: 

R[write to console]: downloaded 808 KB


R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/farver_2.1.1

...

** help
*** installing help indices
*** copying figures
** building package indices
** testing if installed package can be loaded from temporary location


During startup - Warning messages:
1: Setting LC_TIME failed, using "C" 
2: Setting LC_MESSAGES failed, using "C" 
3: Setting LC_MONETARY failed, using "C" 


** testing if installed package can be loaded from final location


During startup - Warning messages:
1: Setting LC_TIME failed, using "C" 
2: Setting LC_MESSAGES failed, using "C" 
3: Setting LC_MONETARY failed, using "C" 


** testing if installed package keeps a record of temporary installation path
* DONE (modelr)
During startup - Warning messages:
1: Setting LC_TIME failed, using "C" 
2: Setting LC_MESSAGES failed, using "C" 
3: Setting LC_MONETARY failed, using "C" 
ERROR: dependencies ‘rvest’, ‘xml2’ are not available for package ‘tidyverse’
* removing ‘/opt/homebrew/lib/R/4.3/site-library/tidyverse’
R[write to console]: 

R[write to console]: 
R[write to console]: The downloaded source packages are in
	‘/private/var/folders/t_/62401zpn7dj6xlnzf_bjdsb00000gn/T/RtmpQY9ttV/downloaded_packages’
R[write to console]: 
R[write to console]: 



In [5]:
TCGAutils = importr('TCGAutils')

PackageNotInstalledError: The R package "TCGAutils" is not installed.

In [ ]:
###############################################################################################################
##################################### Initialisation ##########################################################
###############################################################################################################

In [2]:
import os
from pymongo import MongoClient
import bycon
import pandas as pd
import time, base36

# Prepare mongodb tools
client = MongoClient()
db = client.progenetix
bs = db.biosamples

In [3]:
df = pd.read_csv('../temp/mappingfile.tsv', sep = '\t')

In [16]:
df.keys()

Index(['biosample_id', 'variant_id', 'callset_id', 'chromosome', 'start',
       'end', 'strand', 'reference_bases', 'alternate_bases', 'hgvsc',
       'variant_classification', 'variant_type', 'hgvsp', 'hgvsp_short',
       'aliquot_id', 'reference_id', 'case_id', 'sample_id'],
      dtype='object')

In [14]:
dd_filtered = df.loc[df['variant_type'] == 'SNP', ['hgvsc', 'reference_bases',
                                                   'alternate_bases', 'start', 'end']].copy()
INS = df.loc[df['variant_type'] == 'INS', ['start', 'end', 'reference_bases', 'alternate_bases']].copy()
# Drop rows with missing values in HGVSC
dd_filtered.dropna(subset=['hgvsc'], inplace=True)

# Extract the last character from HGVSC and assign it to a new column 'HGVSC_ref_base'
dd_filtered['HGVSC_ref_base'] = dd_filtered['hgvsc'].str.split('>').str[0].str[-1]
dd_filtered['HGVSC_alt_base'] = dd_filtered['hgvsc'].str.split('>').str[1]


# Reset the index if necessary
dd_filtered.reset_index(drop=True, inplace=True)

compare = dd_filtered[['hgvsc', 'start', 'end', 'HGVSC_ref_base', 'reference_bases', 'HGVSC_alt_base', 'alternate_bases']]
issues = compare[compare['reference_bases'] != compare['HGVSC_ref_base']]
correct = compare[compare['reference_bases'] == compare['HGVSC_ref_base']]

In [15]:
correct

,hgvsc,start,end,HGVSC_ref_base,reference_bases,HGVSC_alt_base,alternate_bases
1,c.103G>C,226064454,226064454,G,G,C,C
2,c.658A>T,26279162,26279162,A,A,T,T
3,c.1380C>A,31266476,31266476,C,C,A,A
4,c.3484C>A,98303765,98303765,C,C,A,A
5,c.2510T>A,124764127,124764127,T,T,A,A
...,...,...,...,...,...,...,...
2424419,c.315G>T,106612395,106612395,G,G,T,T
2424420,c.813G>A,114906851,114906851,G,G,A,A
2424422,c.165T>A,139537086,139537086,T,T,A,A
2424423,c.56G>T,153536303,153536303,G,G,T,T
